In [60]:
import time
import xml.etree.ElementTree as ET

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import seaborn as sns
from IPython.display import HTML, display
from matplotlib.cm import ScalarMappable
from matplotlib.colors import LinearSegmentedColormap, Normalize
from matplotlib.patches import Patch
from tqdm import tqdm

display(HTML("<style>.container { width:100% !important; }</style>"))

In [38]:
# Journal impact factors: Source: https://www.scimagojr.com/journalrank.php
j_list = pd.read_csv("scimagojr 2022.csv", sep=";")
j_list.head()

,Rank,Sourceid,Title,Type,Issn,SJR,SJR Best Quartile,H index,Total Docs. (2022),Total Docs. (3years),...,Total Cites (3years),Citable Docs. (3years),Cites / Doc. (2years),Ref. / Doc.,Country,Region,Publisher,Coverage,Categories,Areas
0,1,28773,Ca-A Cancer Journal for Clinicians,journal,"15424863, 00079235","86,091",Q1,198,44,118,...,30318,85,"299,99","97,00",United States,Northern America,Wiley-Blackwell,1950-2022,Hematology (Q1); Oncology (Q1),Medicine
1,2,29431,Quarterly Journal of Economics,journal,"00335533, 15314650","36,730",Q1,292,36,122,...,2141,122,"14,83","66,61",United Kingdom,Western Europe,Oxford University Press,1886-2022,Economics and Econometrics (Q1),"Economics, Econometrics and Finance"
2,3,20315,Nature Reviews Molecular Cell Biology,journal,"14710072, 14710080","34,201",Q1,485,121,328,...,13331,156,"35,47","89,29",United Kingdom,Western Europe,Nature Publishing Group,2000-2022,Cell Biology (Q1); Molecular Biology (Q1),"Biochemistry, Genetics and Molecular Biology"
3,4,18434,Cell,journal,"00928674, 10974172","26,494",Q1,856,420,1637,...,67791,1440,"43,80","65,74",United States,Northern America,Cell Press,1974-2022,"Biochemistry, Genetics and Molecular Biology (...","Biochemistry, Genetics and Molecular Biology"
4,5,15847,New England Journal of Medicine,journal,"00284793, 15334406","26,015",Q1,1130,1410,4561,...,133956,1854,"33,93","10,21",United States,Northern America,Massachussetts Medical Society,1945-2022,Medicine (miscellaneous) (Q1),Medicine


In [40]:
def search_pubmed(query, max_results=None):
    base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
    params = {"db": "pubmed", "term": query, "retmax": max_results}

    if max_results:
        params["retmax"] = max_results

    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        root = ET.fromstring(response.content)
        pmids = [id_elem.text for id_elem in root.findall("IdList/Id")]
        return pmids
    elif response.status_code == 429:
        time.sleep(2)
        return search_pubmed(query, 300)
    else:
        print("Error: Unable to fetch data from PubMed")
        return []

In [41]:
def fetch_details(pmids):
    ids = ",".join(pmids)
    base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi"
    params = {"db": "pubmed", "retmode": "xml", "id": ids}

    response = requests.get(base_url, params=params)
    if response.status_code != 200:
        print("Error: Unable to fetch details from PubMed")
        return None

    return response.content

In [42]:
def parse_pubmed_efetch(xml_data):
    root = ET.fromstring(xml_data)

    articles = []
    for article in root.findall(".//PubmedArticle"):
        title = article.find(".//ArticleTitle").text

        journal_title = article.find(".//Journal/Title").text

        authors = []
        for author in article.findall(".//Author"):
            lastname = author.find("LastName")
            firstname = author.find("ForeName")
            if lastname is not None and firstname is not None:
                authors.append(f"{firstname.text} {lastname.text}")

        pmcid = None
        for article_id in article.findall(".//ArticleId"):
            if article_id.get("IdType") == "pmc":
                pmcid = article_id.text
                break

        articles.append(
            {
                "title": title,
                "journal": journal_title,
                "authors": authors,
                "pmcid": pmcid,
            }
        )

    return articles

In [43]:
input_data = pd.read_csv("drug_cell_gene.csv.gz", index_col=0)
attention = pd.read_csv("attention.csv.gz", index_col=0)
attention.index = input_data.index
attention.columns = input_data.columns
attention

/var/folders/s4/6xgylm7x3yzgfsk6sbvk_7_c0000gn/T/ipykernel_73124/2706577698.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  input_data = pd.read_csv('drug_cell_gene.csv.gz', index_col=0)
/var/folders/s4/6xgylm7x3yzgfsk6sbvk_7_c0000gn/T/ipykernel_73124/2706577698.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  attention = pd.read_csv('attention.csv.gz', index_col=0)


,740,752,755,762,1390,1895,3053,3088,6396,8806,...,ZNF207,ZNF22,ZNF580,ZNF593,ZNF706,ZNHIT1,ZNHIT3,ZNRD1,ZWINT,ZYX
740,0.000404,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.003412,0.003558,0.003408,0.003410,0.003498,0.003412,0.003494,0.003540,0.003394,0.003387
752,0.000000,0.000404,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.003327,0.003505,0.003340,0.003308,0.003361,0.003338,0.003412,0.003434,0.003295,0.003294
755,0.000000,0.000000,0.000404,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.003307,0.003430,0.003286,0.003286,0.003371,0.003272,0.003372,0.003386,0.003289,0.003258
762,0.000000,0.000000,0.000000,0.000404,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.003330,0.003448,0.003285,0.003300,0.003380,0.003285,0.003378,0.003418,0.003271,0.003272
1390,0.000000,0.000000,0.000000,0.000000,0.000404,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.003382,0.003535,0.003351,0.003374,0.003419,0.003370,0.003442,0.003498,0.003341,0.003323
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZNHIT1,0.000404,0.000319,0.000413,0.000323,0.000405,0.000415,0.000411,0.000313,0.000312,0.000314,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000404,0.000000,0.000000,0.000000,0.000000
ZNHIT3,0.000419,0.000321,0.000419,0.000320,0.000415,0.000422,0.000421,0.000312,0.000313,0.000314,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000404,0.000000,0.000000,0.000000
ZNRD1,0.000407,0.000317,0.000414,0.000319,0.000407,0.000416,0.000411,0.000313,0.000312,0.000314,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000404,0.000000,0.000000
ZWINT,0.000398,0.000316,0.000404,0.000315,0.000402,0.000411,0.000408,0.000309,0.000305,0.000310,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000404,0.000000


In [44]:
att_dg = attention.iloc[:269, 329:]
conv = dict(pd.read_csv("nsc_dti.csv", index_col=0)[["NSC", "Drug Name"]].values)

In [45]:
results = []
pmid_list = []
for drug in tqdm(att_dg.index):
    genes = list(att_dg.loc[drug].sort_values(ascending=False).head(5).index)
    if drug == 759263:
        print(genes)

    for gene in genes:

        d = conv[int(drug)]
        if d == "benzaldehyde (BEN)":
            d = "benzaldehyde"
        elif "Derivative" in d:
            d = d[:-11]

        if "ADA" == gene:
            gene = "adenosine deaminase"

        pmids = search_pubmed("{} AND {}".format(d, gene), max_results=300)
        pmid_list.append([drug, gene, pmids])
        if pmids != None:
            details = fetch_details(pmids)
            if details != None:
                details = details.decode("utf-8")
                res = pd.DataFrame(parse_pubmed_efetch(details))
                if len(res) > 0:
                    results.append(
                        [
                            drug,
                            conv[int(drug)],
                            gene,
                            len(pmids),
                            len(
                                (
                                    res[
                                        res.journal.str.lower().isin(
                                            set(j_list["Title"].str.lower())
                                        )
                                    ]
                                )
                            ),
                        ]
                    )
                else:
                    results.append([drug, conv[int(drug)], gene, len(pmids), 0])
            else:
                results.append([drug, conv[int(drug)], gene, len(pmids), 0])
        else:
            results.append([drug, conv[int(drug)], gene, 0, 0])

  0%|          | 0/269 [00:00<?, ?it/s]

Error: Unable to fetch details from PubMed


  0%|          | 1/269 [00:05<26:07,  5.85s/it]

Error: Unable to fetch details from PubMed
Error: Unable to fetch details from PubMed
Error: Unable to fetch details from PubMed


  1%|          | 2/269 [00:13<30:31,  6.86s/it]

...

Error: Unable to fetch details from PubMed
Error: Unable to fetch details from PubMed
Error: Unable to fetch details from PubMed


100%|█████████▉| 268/269 [29:37<00:06,  6.73s/it]

Error: Unable to fetch details from PubMed
Error: Unable to fetch details from PubMed


100%|██████████| 269/269 [29:43<00:00,  6.63s/it]

Error: Unable to fetch details from PubMed


In [48]:
results = pd.DataFrame(results)
results.columns = ["NSC", "drug", "gene", "num", "num in journals"]
results

,NSC,drug,gene,num,num in journals
0,740,METHOTREXATE,ATIC,58,44
1,740,METHOTREXATE,TYMS,65,0
2,740,METHOTREXATE,DHFR,300,210
3,740,METHOTREXATE,MIR330,0,0
4,740,METHOTREXATE,SNORD16,0,0
...,...,...,...,...,...
1340,791785,Sapacitabine,PGR,0,0
1341,791785,Sapacitabine,ZAP70,0,0
1342,791785,Sapacitabine,adenosine deaminase,0,0
1343,791785,Sapacitabine,EPHB6,0,0


In [49]:
results.to_csv("result_all.csv", index=False)